# Image denoising example (not working yet)

In [ ]:
# %matplotlib qt
# from matplotlib import rc
# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
# rc('text', usetex=True)

In [ ]:
import sys
sys.path.append("../")
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
from joblib import Parallel, delayed
from skimage.color import rgb2gray
from skimage.util import random_noise
from mmhuber.mmalgorithm import hubniht

In [ ]:
# Reading data
I = plt.imread('../data/lena_std.tif')
I = rgb2gray(I)
I = I[::2,::2]

# Adding noise
I_noisy = random_noise(I, 's&p', amount=0.05)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(9, 6))
axes[0].imshow(I, cmap='gray')
axes[0].title.set_text('Original image')
axes[0].axis('off')
axes[1].imshow(I_noisy, cmap='gray')
axes[1].title.set_text('Noisy image')
axes[1].axis('off')

In [ ]:
# Simulation parameters
blocksize = 8 # Do not change as it depends on the dictionary which is generated using matlab
K = 2 # Sparsity value
c = 1.345 # Huber's constant

# Loading dictionary
X = np.load('../data/dictionary.npy')

In [ ]:
# Function to iterate over a slice of image to compute the denoising
def iterate_image_denoising(image_slice, m_r, m_c, c, K , X):
    n_r, n_c = image_slice.shape
    denoised_image = np.zeros((n_r, n_c))
    for i_r in trange(0, n_r-m_r+1): # Iterating rows
        for i_c in range(0, n_c-m_c+1): # Iterating columns
            # Obtaining local data
            y = image_slice[i_r:i_r+m_r, i_c:i_c+m_c].reshape((m_r*m_c,1))

            # Do denoising
            beta, sigma, support = hubniht(y, X, c, K, check_decreasing=True)
            y_hat = X@beta
            y_hat = y_hat.reshape((m_r, m_c))
            denoised_image[i_r:i_r+m_r, i_c:i_c+m_c] = y_hat

    return denoised_image

I_denoised = iterate_image_denoising(I_noisy, blocksize, blocksize, c, K , X)

In [ ]:
plt.figure()
plt.imshow(I_denoised, cmap='gray')
plt.axis('off')

In [ ]:
# # Parallel version still in work...

# # Splitting the image in several small to use parallel processing 
# # (choose so that the splitting is even and the size of each slice is a multiple of the blocksize)
# number_of_threads_columns = 4
# number_of_threads_rows = 2
# image_slices_list = []
# slice_rows_list = np.array_split(I_noisy, number_of_threads_rows, axis=0)
# for image_slice_row in slice_rows_list:
#     image_slices_list.append( np.array_split(image_slice_row, number_of_threads_columns, axis=1)  )

# # Visualize the slices
# fig, axes = plt.subplots(nrows=number_of_threads_rows, ncols=number_of_threads_columns, figsize=(6, 6))
# for i in range(number_of_threads_rows):
#     for j in range(number_of_threads_columns):
#         axes[i,j].imshow(image_slices_list[i][j], cmap='gray')
#         axes[i,j].axis('off')


# # Computing the denoising for each slice in parallel
# results_list = Parallel(n_jobs=number_of_threads_columns*number_of_threads_rows)(
#                     delayed(iterate_image_denoising)(image_slices_list[i][j], blocksize, blocksize, c, K, X) 
#                     for i in range(number_of_threads_rows) 
#                     for j in range(number_of_threads_columns))

# # Putting back the image together from the slices
# I_denoised = []
# for i in range(number_of_threads_rows):
#     I_denoised.append(np.hstack(results_list[i*number_of_threads_columns:(i+1)*number_of_threads_columns]))
# I_denoised = np.vstack(I_denoised)

In [ ]:
# plt.figure()
# plt.imshow(I_denoised, cmap='gray')
# plt.axis('off')